In [ ]:
from osgeo import gdal
from math import floor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## A Notebook used to tile an entire SAR image into distinct footprints to pass into the trained model

In [ ]:
# Opening the raster
raster = "D:\ICEEYE\Proposal_ID_PP0091278\CS-13801\SLEA_3279211_180312\SLEA-Repojected-WSG84.tiff"
ds = gdal.Open(raster)

raster_band = ds.GetRasterBand(1)
xoffset, px_w, rot1, yoffset, rot2, px_h = ds.GetGeoTransform() # The transformation matrix to get the coordinates from the picture

icesat_data = "D:\ICEEYE\Proposal_ID_PP0091278\CS-13801\SLEA_3279211_180312\Coincident-Readings.csv"
icesat_data = pd.read_csv(icesat_data)
num_rows = len(icesat_data)

print("Projection: ", ds.GetProjection()) # get projection
print("Tiff width (pixels):", ds.RasterXSize)  # number of columns
print("Tiff Height (pixels):", ds.RasterYSize)  # number of rows
print("Band count:", ds.RasterCount)  # number of bands
print("GeoTransform Matrix", ds.GetGeoTransform())
print("Statistics:", ds.GetRasterBand(1).GetStatistics(0,1))
print(f"Number of data points: {num_rows}")
print(icesat_data.head(1))

data_array2 = ds.GetRasterBand(1).ReadAsArray() # Indexing for raster bands starts at 1, not 0

# plt.figure(figsize=(10, 10))
# plt.imshow(data_array2, cmap="viridis") # Data array contains elevation data from the raster band. Is this always true?
# plt.colorbar()
# plt.show()

In [ ]:
# Utility Functions
def getTileCenter(index: int):
    row = icesat_data.iloc[[index]]
    lat = float(row['Lat'])
    lon = float(row['Lon'])
    pixel_x = (lon - xoffset) / px_w
    pixel_x = floor(pixel_x - 8.5)

    pixel_y = (lat - yoffset) / px_h
    pixel_y = floor(pixel_y - 8.5)

    calculatedCoord = (pixel_x, pixel_y)
    return calculatedCoord

def getFileDirectoryAndName(value: float, index: int, beam_name: str):
    path = "D:\\ICEEYE\\Proposal_ID_PP0091278\\CS-13801\\SLEA_3279211_180312\\Ice-Sat-Tiles-Scaled"
    return path + "\\" + beam_name + str(index) + "-" + str(value) + ".tif"

def isTileEmpty(pixelTuples: tuple):
    topLeftX, topLeftY = pixelTuples
    tile = raster_band.ReadAsArray(topLeftX,topLeftY, 17, 17)
    if tile is not None and tile.any():
        sortList = sorted(tile[0])
        # print([sortList[0], sortList[len(sortList)-1]])
        return True
    return False

In [ ]:
print("Tiff width (pixels):", ds.RasterXSize)  # number of columns
print("Tiff Height (pixels):", ds.RasterYSize)  # number of rows
FOOTPRINT_WIDTH = 17
VISUAL_FACTOR = 10

x_corners = []
y_corners = []

tile_num = 0
y_index = 0
measuredVal = 0
numRepeats = 0
for i, row in icesat_data.iterrows():
    pixelCoords = getTileCenter(i)
    print(f"{i} / {num_rows}")
    if i + 1 < num_rows:
        nextCoords = getTileCenter(i+1)
    else:
        nextCoords = (False, False)
    beam_name = row['Beam']
    fb_h = row['fbh']
    window = (*pixelCoords, 17, 17)
    # If the next point is at the same pixel coordinate, just update the running total
    if pixelCoords == nextCoords:
        measuredVal += fb_h
        numRepeats += 1
    else:
        # otherwise, tile the image using the average of the running total's freeboard measurement.
        if numRepeats == 0:
            tileName = getFileDirectoryAndName(fb_h, i, beam_name)
        else:
            measuredVal += fb_h
            numRepeats += 1
            tileName = getFileDirectoryAndName(measuredVal / numRepeats, i, beam_name)
        hasData = isTileEmpty(pixelCoords)
        if not hasData:
            print(i, "Has no data", pixelCoords)
        else:
            if i % VISUAL_FACTOR == 0:
                x, y = pixelCoords
                x_corners.append(x)
                y_corners.append(y)
                print(i, " - ", tileName)
            gdal.Translate(tileName, raster, srcWin = window, format="PNG", scaleParams=[[]])
            tile_num += 1
            measuredVal = 0
            numRepeats = 0

In [ ]:
plt.imshow(data_array2) # Data array contains elevation data from the raster band. Is this always true?
plt.scatter(x=x_corners, y=y_corners, c='r', s=1, alpha=0.5)
plt.title(f"SAR Tiling for Trained Model (Factor of {VISUAL_FACTOR})")
plt.figtext(0.9, 0.25, f'Total of {tile_num} tiles', horizontalalignment='right')
plt.show()